# Préparer le dataset

In [1]:
raw_data_folder = './data/genres_original/'

# take all folder in raw_data_folder
import os
genres = os.listdir(raw_data_folder)

print(genres)

['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']


In [2]:
from pydub import AudioSegment
import os

def split_wav_file(input_file_path, output_folder):
    # Vérifier si le dossier de sortie existe, sinon le créer
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Vérifier si le fichier  de clip existe, si oui return
    if os.path.exists(os.path.join(output_folder, "clip_1.wav")):
        return
    

    try: 
        sound = AudioSegment.from_wav(input_file_path)

        clip_duration = 3 * 1000  # Durée de chaque clip en millisecondes
        total_duration = len(sound)

        total_clips = total_duration // clip_duration

        for i in range(total_clips):
            start_time = i * clip_duration
            end_time = (i + 1) * clip_duration
            clip = sound[start_time:end_time]
            clip.export(os.path.join(output_folder, f"clip_{i + 1}.wav"), format="wav")
    except Exception as e:
        print(e)

tree_seconds_folder = './data/genres_3s/'

for genre in genres:
    # create a folder for each genre,
    # then take all the music files in the genre folder
    # and separate them into 3s clips
    genre_folder = tree_seconds_folder + genre
    if not os.path.exists(genre_folder):
        os.mkdir(genre_folder)

    genre_path = raw_data_folder + genre
    music_files = os.listdir(genre_path)

    for music_file in music_files:
        music_file_path = genre_path + '/' + music_file
        # check if file exist
        if os.path.exists(music_file_path):
            split_wav_file(music_file_path, genre_folder)

c:\Users\fcola\Documents\COURS\opendata\music_genre_classification\.venv\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [14]:
import librosa
import pandas as pd
import numpy as np

def extract_audio_features(file_path, label):
    try:
        sr = 44100
        # Chargement du fichier audio
        y, sr = librosa.load(file_path, sr=sr)

        # spectrogram, tempo, chroma, mfccs, spectral_contrast

        spectrogram = np.abs(librosa.stft(y))
        spectrogram_min = np.min(spectrogram)
        spectrogram_max = np.max(spectrogram)
        spectrogram_mean = np.mean(spectrogram)
        spectrogram_std = np.std(spectrogram)

        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_min = np.min(chroma)
        chroma_max = np.max(chroma)
        chroma_mean = np.mean(chroma)
        chroma_std = np.std(chroma)

        mfccs = librosa.feature.mfcc(y=y, sr=sr)
        mfccs_min = np.min(mfccs)
        mfccs_max = np.max(mfccs)
        mfccs_mean = np.mean(mfccs)

        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        spectral_contrast_min = np.min(spectral_contrast)
        spectral_contrast_max = np.max(spectral_contrast)
        spectral_contrast_mean = np.mean(spectral_contrast)
        spectral_contrast_std = np.std(spectral_contrast)

        audio_features = {
            'label': label,
            'spectrogram_min': spectrogram_min,
            'spectrogram_max': spectrogram_max,
            'spectrogram_mean': spectrogram_mean,
            'spectrogram_std': spectrogram_std,
            'tempo': tempo,
            'chroma_min': chroma_min,
            'chroma_max': chroma_max,
            'chroma_mean': chroma_mean,
            'chroma_std': chroma_std,
            'mfccs_min': mfccs_min,
            'mfccs_max': mfccs_max,
            'mfccs_mean': mfccs_mean,
            'spectral_contrast_min': spectral_contrast_min,
            'spectral_contrast_max': spectral_contrast_max,
            'spectral_contrast_mean': spectral_contrast_mean,
            'spectral_contrast_std': spectral_contrast_std
        }
        return audio_features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# # test extract_audio_features pour 1 fichier
# file_test = './data/genres_3s/blues/clip_1.wav'

# extract_audio_features(file_test, 'blues')

In [15]:
df_list = []

for genre in genres:
    wav_folder = os.path.join(tree_seconds_folder, genre)
    wav_files = [os.path.join(wav_folder, file) for file in os.listdir(wav_folder) if file.endswith('.wav')]

    for wav_file in wav_files:
        audio_features = extract_audio_features(wav_file, genre)
        if audio_features:
            df_list.append(audio_features)
        else:
            print(f"Skipping {wav_file} due to processing error.")
        
    print('Done for genre: ', genre, ' with ', len(wav_files), ' files')

df = pd.DataFrame(df_list, columns=['label', 'spectrogram_min', 'spectrogram_max', 'spectrogram_mean', 'spectrogram_std', 'tempo', 'chroma_min', 'chroma_max', 'chroma_mean', 'chroma_std', 'mfccs_min', 'mfccs_max', 'mfccs_mean', 'spectral_contrast_min', 'spectral_contrast_max', 'spectral_contrast_mean', 'spectral_contrast_std'])
df.to_csv('./clean_data/audio_features_3s.csv', index=False)

Done for genre:  blues  with  10  files
Done for genre:  classical  with  10  files
Done for genre:  country  with  10  files
Done for genre:  disco  with  10  files
Done for genre:  hiphop  with  10  files
Done for genre:  jazz  with  10  files
Done for genre:  metal  with  10  files
Done for genre:  pop  with  10  files
Done for genre:  reggae  with  10  files
Done for genre:  rock  with  10  files
